<h1>Yelp Data Exploration</h1>

Remove logs

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns

import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from pyspark.sql.functions import *

from pyspark.sql import *
%matplotlib inline

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext

conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.driver.cores", "3")
         .set("spark.executor.memory", "4g"))
sc = SparkContext(conf = conf)

sqlContext = HiveContext(sc)

In [3]:
# remove spark heavy logging
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

<h1>Import Data</h1>

In [4]:
# HDFS Base path for Yelp Data
base_path = '/home/jovyan/work/Documents/F3B_ISA/Yelp/yelp/json'

In [5]:
# Yelp file types list
file_type_list = ['businesses', 'checkins', 'reviews', 'tips', 'users']


In [6]:
# Dataframe dictionnary for each file type
dataframe_map = {t: sqlContext.read.json(base_path + '/%s' % t)
                 for t in file_type_list}


AttributeError: 'list' object has no attribute 'show'

<h1>Most friends user as Yelper Star</h1>

In [14]:
most_friends_reviews = df_reviews_vegas.where(df_reviews_vegas.user_id.isin(most_friends_user)).collect()

In [15]:
len(most_friends_reviews)

31

In [16]:
results = pd.DataFrame(columns=['user_id', 'review_id', 'prop_reviews'])

def getReviewsProportions(row):
    
    #row = yelper_star_reviews.where(yelper_star_reviews.review_id == review_id).collect()[0]
    start_date = row.date
    business_id = row.business_id
    end_date = start_date + datetime.timedelta(days=30)
    #Count the reviews
    count_reviews_before = df_reviews.where((df_reviews.business_id == business_id)
                                    & (df_reviews.date <= start_date)).count()
    count_reviews_after_month = df_reviews.where((df_reviews.business_id == business_id)
                                    & (df_reviews.date < end_date)).count()
    
    
    prop = 100*(count_reviews_after_month - count_reviews_before)/count_reviews_before
    index = len(results.index)
    results.set_value(index, 'user_id', row.user_id)
    results.set_value(index, 'review_id', row.review_id)
    results.set_value(index, 'prop_reviews', prop)

In [ ]:
for row in most_friends_reviews :
    getReviewsProportions(row)

open('_FINISHED', 'a').close()

In [ ]:
sns.boxplot(y='prop_reviews', data=results)

In [ ]:
results.prop_reviews.hist(bins=100)